In [1]:
import os, os.path
os.environ["CUDA_VISIBLE_DEVICES"]="0"
tf_device='/gpu:0'
import random
import pickle
import numpy as np
import math
import matplotlib.pyplot as plt
import time
import tensorflow as tf
tf.get_logger().setLevel('INFO')
tf.keras.backend.floatx()
from collections import deque
import itertools
from sklearn.utils import shuffle
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense
from libs.utils import *
from libs.generate_boxes import *
from libs.dqn import *


num_episode = 10000
global_step = 0
tr_l, h_fill, tr_r,avg_loss_l,history_eps,used_boxes_eps  = [],[],[],[],[],[]
K = 1
n_candidates = 5
num_max_remain = 180 
num_bbox_type = 20 #사용할 대박스 수
sc_target = 64 #최대 대박스 크기를 sc_target 크기만큼 줄임

mi = 'U1'
mbox_type = 'PP'
f_name = 'plan_predict_input/' + mi+'_'+mbox_type
output_col = ['pack_no','creat_dd', 'odr_no', 'ctmmny_cd','vctp_cd', 'prod_no', 'mbox_cd', 'bbox_cd',
              'B_CBM',  'mbox_acctc_qty', 'bbox_acctc_qty', 
              'packing_rate_volume', 'packing_rate_cbm','x','y','z']

# 대박스 규격 로드
df_bbox = pd.read_csv('data/preprocessed/bbox/' + mi + '_' + mbox_type + '.csv')
casemaster=pd.read_csv('data/preprocessed/casemaster.csv')
df_bbox = df_bbox.iloc[:20]
df_bbox = pd.merge(df_bbox, casemaster[['CD', 'CBM']], how='left', left_on=['CASE_CD'], right_on=['CD']).drop(['CD'],axis=1)
df_bbox = df_bbox.drop(['CASE_CD_count'], axis=1)
# 대박스 규격 로드 (Env)
bbox_size = pd.read_csv('data/preprocessed/bbox/' + mi + '_' + mbox_type + '.csv')
bbox_size = bbox_size.values[:num_bbox_type, 1:-1]
bbox_size = np.concatenate([np.sort(bbox_size[:,:2], axis=1), bbox_size[:,2:]], axis=-1)
# 대박스 스케일링
sc = sc_target/np.max(bbox_size, axis =0)[:2] 
bbox_sc = bbox_size.copy() 
bbox_sc[:, :2] = bbox_size[:,:2] * sc # 세로가로만 스케일링
bbox_sc = (bbox_sc+0.5).astype('int') #반올림
mxl, mxb, mxh = np.max(bbox_sc, axis =0 ) # 최대 규격
print(mxl, mxb, mxh)
# 대박스 행렬로 변환
bbox_type = [] #np.zeros((1, 20, 20))
for l,b,h in bbox_sc:
    bbox = np.zeros((mxl, mxb))
    bbox[l:,:] = mxh
    bbox[:,b:] = mxh
    bbox[:l,:b] = mxh - h
    bbox_type.append(bbox)
bbox_type = np.stack((bbox_type)) #(5, 20, 20)

# 중박스 데이터 로드 
group_key = ['creat_dd', 'odr_no', 'ctmmny_cd', 'mbox_type']
df = pd.read_csv('data/preprocessed/'+f_name +'_resid.csv')
org_col = df.columns
df = df.sort_values('sum_wt', ascending = False)
df = df.set_index(group_key).sort_index()
group_idx = df.index.drop_duplicates()
# 중박스 데이터 로드 (입력 데이터)
with open('data/preprocessed/' + f_name +'_resid.pickle', 'rb') as handle: 
    boxes_multi =  pickle.load(handle)
with open('data/preprocessed/' + f_name +'_resid_v0.pickle', 'rb') as handle: 
    boxes_multi_n =  pickle.load(handle)

boxes_multi = [ np.concatenate([np.sort(x[:, :2], axis=1), x[:, 2:]], axis=-1) for x in  boxes_multi]
boxes_multi_n = [ np.concatenate([np.sort(x[:, :2], axis=1), x[:, 2:]], axis=-1) for x in  boxes_multi_n]

# 중박스 스케일링
boxes_multi_sc = [ np.concatenate([np.ceil(x[:, :2] * sc), x[:, 2:3]], axis=-1).astype('int') for x in  boxes_multi]
boxes_multi_n_sc = [ np.concatenate([np.ceil(x[:, :2] * sc), x[:, [2,5]]], axis=-1).astype('int') for x in  boxes_multi_n]
cbm_per_mbox = [x[:,-1] for x in boxes_multi]

# 중박스 크기를 행렬로 표현
max_size = sc_target
boxes_mat_multi = np.stack([size2mat(x, num_max_remain, max_size) for x in boxes_multi_n_sc]) # (N, num_max_remain, max_size, 2)


# Environment
env = Bpp3DEnvMS(length = mxl, breadth = mxb, height = mxh, bbox_type = bbox_type)
# Agent
agent = DQNAgent( L = mxl, B = mxb , H = mxh, n_remains = num_max_remain, n_loading=K, max_size = max_size,lr=1e-4, exp_steps=9000, 
                 train_st = 1500, memory_len=2000, update_target_rate = 1000, net = 'DDQN_DNN') #DDQN_CNNDNN

64 64 1320
distribution DDQN_DNN


In [ ]:
is_pred =  False #False #

if is_pred:
    num_episode = len(boxes_multi_sc)
    # 예측 결과 테이블
    pk = pd.DataFrame(columns = ['pack_no']+list(org_col)+['packing_rate_cbm','packing_rate_volume','x','y','z'])
    pack_no = 0
    agent.epsilon = 0

for e in range(num_episode): #
    st=time.time()
    step = 0
    
    if is_pred:
        df_p = df.loc[group_idx[e]].reset_index().copy()
        df_p = df_p.loc[df_p.index.repeat(df_p['num_mbox'])] # 각 행을 num_mbox번 만큼 반복
        
    else:
        if e > 9000:
            agent.epsilon = 0
        if agent.epsilon > agent.epsilon_end and len(agent.memory)>=agent.train_start:
            agent.epsilon -= agent.epsilon_decay_step
            
    ith_data = e % len(boxes_multi)#5#
    boxes_all_sc = boxes_multi_sc[ ith_data].copy().astype('int')# 
    cbm_all = cbm_per_mbox[ith_data].copy()
    r_boxes = boxes_all_sc.copy()
    r_mat = boxes_mat_multi[ith_data].copy() #(50,64,2)
    boxes_idx = np.arange(len(boxes_all_sc))
    loaded_idx = []
    used_boxes, pred_pos = [],[]
    env_rewards, is_last, used_bbox = [], [], []
    history, h_load, h_remain_size, h_load_size = [],[],[],[]
    t_history, t_load, t_remain_size, t_load_size = [],[],[],[]
    
    while len(r_boxes)>0: #모든 중박스를 다 적입할 때까지 반복 (새로운 대박스 시작)
        done = False
        len_group = 0
        cbm_reward = 0
        # state 선택
        in_state = np.array( [ np.concatenate([np.zeros((env.length,env.breadth,2)), b[...,np.newaxis]], axis=-1) for b in bbox_type] )#(5,20,20,3)
        in_state = (in_state/env.height).astype(np.float32)
        loading_loc_c = np.zeros((num_bbox_type, env.length, env.breadth, K)) # 적재 위치
        in_loading = np.zeros((num_bbox_type, K, max_size, 2))
        in_remain = np.array([r_mat] * num_bbox_type).astype(np.float32)
        # 대박스 규격 선택
        action_idx = agent.get_action(in_state, loading_loc_c, in_remain, in_loading)
        env.reset(action_idx)
        history.append(in_state[action_idx])
        h_load.append(loading_loc_c[action_idx])
        h_remain_size.append(in_remain[action_idx])
        h_load_size.append(in_loading[action_idx])
        
        while not done:#중박스에 대해 반복##############
            state = env.container_s.copy()
            state_h = env.container_h.copy()
            global_step += 1
            step += 1
            cbm_all_ratio = cbm_all/df_bbox.iloc[env.bbox_idx]['CBM']
            # 적재 위치 계산 ########################
            k = min(K, len(r_boxes))
            selected, selected_idx = cbn_select_boxes(r_boxes[:n_candidates],boxes_idx[:n_candidates], k) # 조합
            s_order, s_order_idx = get_selected_order(selected, selected_idx, k) #순열
            s_order, s_order_idx = rot_one_order(s_order, s_order_idx) # 회전
            order_idx_c, num_loading_c, loading_idx_c, loading_size_c, loading_xyz_c, loading_loc_c, next_state_c, next_h_c  =\
                get_selected_location(s_order, s_order_idx, state, state_h, env.height,K, cbm_reward ,cbm_all_ratio ) #위치
            
            #print('s_order',len(s_order))
            # action ########################
            if len(order_idx_c) ==0: #<= 1: #action 스킵
                action_idx = 0
            else: # 선택 가능 ->  action 실행
                in_state, in_remain, in_loading = \
                    raw2input(state, state_h, env.bbox, r_mat, num_max_remain, K, s_order[order_idx_c], max_size, env.height) # input 변환 
                action_idx = agent.get_action(in_state, np.array(loading_loc_c), in_remain, in_loading)
                
            # 대박스 선택 후의 target network 입력
            if len_group == 0:#len(t_history) == 0: 
                if np.sum(loading_size_c) == 0:#len(order_idx_c) == 0: # 적입 불가능한 경우
                    t_history.append(np.stack([state,state_h,env.bbox],axis=-1).reshape((1, env.length, env.breadth, 3)))
                    t_load.append(np.zeros((1, env.length, env.breadth, K)) )
                    t_remain_size.append(np.zeros((1, num_max_remain, max_size, 2)))
                    t_load_size.append(np.zeros((1, K, max_size, 2))) 
                    is_last.append(True)
                else: #적입 가능한 경우
                    t_history.append([in_state[action_idx]]) # 적입 전 상태
                    t_load.append(np.array([loading_loc_c[action_idx]])) # 적재입된 상태
                    t_remain_size.append([in_remain[action_idx]]) # 선택 후 전처리된 남은 박스 사이즈
                    t_load_size.append([in_loading[action_idx]]) # 적입된 박스 사이즈
                    is_last.append(False)

            # 다음 step ################################
            if np.sum(loading_size_c) == 0: # 주어진 중박스 중 하나도 적재하지 못한 경우
                r_boxes = get_remain(r_boxes[:n_candidates], r_boxes) # 해당 범위의 박스들 제외
                r_mat = r_mat[n_candidates:,:,:]
                r_mat = np.concatenate([r_mat,np.zeros((n_candidates, max_size, 2))])
                boxes_idx = boxes_idx[n_candidates:]
            else: # 한 가지 이상의 action 옵션이 있는 경우(적재 가능한 경우)
                #print('loading',loading_idx_c[action_idx] , num_loading_c, np.sum(loading_size_c))
                env.step(next_state_c[action_idx], next_h_c[action_idx])  # env의 다음 state
                # index 업데이트
                order_idx = order_idx_c[action_idx]
                loading_idx = loading_idx_c[action_idx]
                loaded_idx += loading_idx # 적입된 박스 인덱스
                boxes_idx = np.setdiff1d(boxes_idx, s_order_idx[order_idx]) # 남은 박스의 인덱스
                #boxes_idx = np.setdiff1d(boxes_idx, loading_idx) 
                # 남은 박스 업데이트
                r_boxes = boxes_all_sc[boxes_idx]
                r_mat = size2mat(np.concatenate([r_boxes, np.ones((len(r_boxes), 1))], axis = -1).astype('int'), num_max_remain, max_size)
                # cbm
                cbm_reward += (np.sum(cbm_all_ratio[loading_idx]))#/df_bbox.iloc[env.bbox_idx]['CBM'])
                # 사용된 박스, 위치 업데이트
                used_boxes = used_boxes + loading_size_c[action_idx] #사용된 박스 업데이트
                pred_pos = pred_pos + loading_xyz_c[action_idx] #예측된 위치 업데이트
                # bbox
                len_group += num_loading_c[action_idx]
            
            
            #남은 박스가 없음 or 높이맵이 모두 채워짐 or cbm 모두 채워짐
            if len(r_boxes) == 0 or np.sum(env.container_h != env.height) == 0 or cbm_reward>=1:
                #if len(r_boxes) == 0 : print('done: no r_boxes')
                if np.sum(env.container_h != env.height) == 0 : print('done: hegith map > h')
                if cbm_reward>=1: print('done: cbm_reward > 1')
                done = True
            
            # action 선택 수가 2 이상이거나 done일 경우 -> append
            if len(num_loading_c) > 1 or done:
                # (s, a) append ########################
                history.append(in_state[action_idx]) # 적재 전 상태
                h_load.append(loading_loc_c[action_idx]) # 적재된 상태
                h_remain_size.append(in_remain[action_idx]) # 선택 후 전처리된 남은 박스 사이즈
                h_load_size.append(in_loading[action_idx]) # 적재된 박스 사이즈
                # (s') append ########################
                t_state = next_state_c[action_idx].copy() # 다음 state 
                t_state_h = next_h_c[action_idx].copy() 
                if done: #마지막 박스까지 채워졌을 경우 or 더 이상 적입할 수 없을 경우
                    in_t_history = np.stack([t_state,t_state_h, env.bbox],axis=-1).reshape((1, env.length, env.breadth, 3))
                    in_t_history = in_t_history/env.height
                    loading_loc_c = np.zeros((1, env.length, env.breadth, K)) 
                    in_t_remains = np.zeros((1, num_max_remain, max_size, 2))
                    in_t_loading = np.zeros((1, K, max_size, 2))
                else: # 다음 state에 대해 action을 취할 수 있는 경우
                    k = min(K, len(r_boxes))
                    selected,tmpidx = cbn_select_boxes(r_boxes[:n_candidates], np.arange(len(r_boxes[:n_candidates])), k) # 조합
                    s_order,tmpidx = get_selected_order(selected, tmpidx, k) # 순열
                    s_order, s_order_idx = rot_one_order(s_order, s_order_idx) # 회전
                    order_idx_c, num_loading_c, loading_idx_c, loading_size_c, loading_xyz_c, loading_loc_c, next_state_c, next_h_c  =\
                        get_selected_location(s_order, tmpidx, t_state, t_state_h, env.height, K, cbm_reward ,cbm_all_ratio )
                    in_t_history, in_t_remains, in_t_loading = \
                        raw2input(t_state, t_state_h, env.bbox, r_mat, num_max_remain, K, s_order[order_idx_c], max_size, env.height)
                    if np.sum(loading_loc_c)==0:
                        loading_loc_c = np.zeros((1, env.length, env.breadth, K))
                    
                t_history.append(in_t_history) # 적재 전 상태
                t_load.append(np.array(loading_loc_c)) # 적재된 상태
                t_remain_size.append(in_t_remains) # 선택 후 전처리된 남은 박스 사이즈
                t_load_size.append(in_t_loading) # 적재된 박스 사이즈
                # (done) append
                is_last.append(done)
                
            if done:
                env_rewards.append(cbm_reward)
                #env_rewards.append(env.terminal_reward())
            if cbm_reward>1: break   
            # 학습 시작
            if len(agent.memory) >= agent.train_start & ~is_pred:
                agent.train_model()
                if global_step % agent.update_target_rate == 0: # target model 업데이트
                    agent.update_target_model()
                avg_loss_l.append(agent.avg_loss/float(step))
        
        boxes_idx = np.setdiff1d(np.arange(len(boxes_all_sc)), loaded_idx)
        #r_boxes = get_remain(used_boxes, boxes_all_sc) # 남은 박스 업데이트: 그룹의 모든 중박스 - 사용된 박스
        r_boxes = boxes_all_sc[boxes_idx]
        if len_group>0: used_bbox.append([env.bbox_idx]*len_group)
        if len_group>0: tr_l.append(cbm_reward)#(env.terminal_reward())
        r_mat = size2mat(np.concatenate([r_boxes, np.ones((len(r_boxes), 1))], axis = -1).astype('int'), num_max_remain, max_size)
        
        if len_group==0: break
        
    # terminal reward 계산 -> replay memory에 추가 
    if done and len(used_boxes)==len(boxes_all_sc):
        # terminal reward 
        a_repeate = 1#6 if env.terminal_reward() ==1.0 else 1
        # append samples
        N = len(history)
        for i in range(N):
            len_remain_group = sum(is_last[i:]) # 남은 포장 대박스 수 (5,4,3,...)
            reward = np.mean(env_rewards[:len_remain_group]) #step reward
            for a in range(a_repeate):
                agent.append_sample(history[i], h_load[i], h_remain_size[i], h_load_size[i], reward, is_last[i],
                                    t_history[i], t_load[i], t_remain_size[i], t_load_size[i])
                
    log = "=====episode:{:5d}| ".format(e)
    log += "memory length:{:5d}| ".format(len(agent.memory))
    log += "epsilon:{:.3f}| ".format(agent.epsilon)
    log += "reward:{:2d}, {:.3f} (".format(len(env_rewards), np.mean(env_rewards))
    log += ', '.join('{:.3f}'.format(k) for k in env_rewards)
    #log += "t_reward: {:.3f} | ".format(t_reward)
    #log += "q avg : {:3.2f} | ".format(agent.avg_q_max / float(step))
    log += ")| avg loss:{:4f} ".format(agent.avg_loss / float(step))
    log += "time:{:.2f}".format(time.time()-st)
    print(log)
    agent.avg_q_max, agent.avg_loss = 0, 0
    
    # 모델 저장
    if e % 5000 == 0:
        agent.model.save_weights("save_model/model", save_format="tf")
    
    if is_pred:
        i = 0
        for p in used_bbox:
            p_loaded_idx = loaded_idx[i:i+len(p)]
            positions =  np.array(pred_pos)[i:i+len(p)]
            positions[:, :2] = positions[:, :2] /sc
            p_bbox = df_bbox.iloc[p[0]]
            p_x = df_p.iloc[p_loaded_idx].copy()
            p_x['pack_no'] = 'resid_'+str(pack_no)
            p_x['packing_rate_cbm'] = (p_x['cbmsum_per_mbox']).sum() / p_bbox['CBM']
            p_x['packing_rate_volume'] = (p_x['M_L']*p_x['M_B']*p_x['M_H']).sum() / (p_bbox['L']*p_bbox['B']*p_bbox['H'])
            p_x['x'] = positions[:, 0]
            p_x['y'] = positions[:, 1]
            p_x['z'] = positions[:, 2]
            pk = pk.append(p_x)

            pack_no += 1
            i += len(p)
    #if cbm_reward>1: break
    #if e>10: break
#print('total training time', time.time()-st)
# if is_pred: pk.to_csv('data/pack/'+mi+'_resid_output.csv',index=False)

In [ ]:
%matplotlib inline

ts_factor = 0.9
df = pd.DataFrame({'value':tr_l})
smooth = df.ewm(alpha=(1 - ts_factor)).mean()
plt.figure(figsize=(20,2))
plt.plot(df["value"],linewidth=0.3)
plt.plot(smooth["value"],linewidth=0.3)
# plt.ylim(-0.2,1.3)
plt.grid()
plt.show

plt.figure(figsize=(20,1.5))
plt.plot(avg_loss_l,'b-',linewidth=0.3)
plt.grid()
plt.show()
plt.show

In [ ]:
print(bbox_sc[env.bbox_idx], bbox_size[env.bbox_idx])
sizes = boxes_multi[ ith_data].copy().astype('int')[loaded_idx][:,:3]#boxes_all[loaded_idx][:,:3]
positions = np.array(pred_pos.copy())
positions[:, :2] = positions[:, :2]/sc

%matplotlib notebook
vis_box(sizes,positions,fs=(3,3),mn=-5, mx=1500)
